In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.feature_extraction import text 
from wordcloud import WordCloud
import re
import string
import nltk
from nltk import pos_tag
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.tokenize import sent_tokenize , word_tokenize
from nltk import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk import pos_tag
from nltk.corpus import stopwords
import contractions

In [2]:
    
stopwords = nltk.corpus.stopwords.words('english')
additional= ['say','like','just','dont','don','im','it','ve','re','we',
                'live','youll','youve','things','thing','youre','right','really','lot',
                'make','know','people','way','day',
                'little', 'maybe','men',"americans","america",
                'kind','heart',"american",'beverybrief','behooves','president','united',"states", "s", "u","evening", "afternoon", "welcome",
               "everybody", "goodbye", 'want', 'take', 'see', 'say', 'go','like','youre','ive','im','really','id','just','dont','didnt','thi','wa',
                  'say','know','make','people',"today",'way','day','time','year','tonight', 'u', 'country', 'well']        
stop_word_file=open('stopwords_file.txt', 'r')
words=stop_word_file.read()

From the data frame we created, the title and content still contain some strings which are not important.
We create the cleanning functions for data pre-processing, removing all the non-important strings from all the speech content, and keep the nouns and verbs for further analysis. 


In [3]:
df_speech=pd.read_csv('obama_speech.csv').iloc[:,1:]
df_speech.head()

,url,content,title,date
0,https://www.americanrhetoric.com/speeches/bara...,"1 Thank you, thank you. My fellow citizens: I...",FIRST PRESIDENTIAL INAUGURAL ADDRESS,20 Jan 2009
1,https://www.americanrhetoric.com/speeches/bara...,We begin this year and this Administration i...,First Presidential Weekly Speech,24 Jan 2009
2,https://www.americanrhetoric.com/speeches/bara...,"Mr. Melhem: Mr. President, thank you for this...",Al-Arabiya Television Interview,26 Jan 2009
3,https://www.americanrhetoric.com/speeches/bara...,"Thank you, Tim, for your hard work on this i...",Executive Compensation Salary Cap Speech,04 Feb 2009
4,https://www.americanrhetoric.com/speeches/bara...,"Good evening, everybody. Please be seated. B...",First Prime Time Press Conference,09 Feb 2009


In [4]:
def ListToString(list):
        string_words = ' '.join(list)
        return string_words

# string to list
def StringToList(string):
        listRes = list(string.split(" "))
        return listRes


In [5]:

def clean_text(text, additional, words, stopwords):
    
        text=re.sub('--', ' ', text)
        text=contractions.fix(text)  #change you're to you are
        text=re.sub('\x92', "'", text)
        text=re.sub('\s', ' ', text)
        text=re.sub("[!@#$';.:]", ' ',text)
        #text= re.sub('\((.*?\))', '', text)
        #text= re.sub('\[.*?\]', '', text)
        #text= re.sub('[%s]' % re.escape(string.punctuation), '',text)
        #text= re.sub('\w*\d\w*', '', text)
        text=re.sub("(\s\d+)","",text)  # remove numbers
        text=re.sub('Book/CDs by Michael E. Eidenmuller, Published by \r\nMcGraw-Hill (2008)',"",text) 
        text=text.lower()
        
        ps = PorterStemmer()
        text=ps.stem(text)
        
        # Remove the punctuations
        tokens = word_tokenize(text)
        # Lower the tokens
        tokens = [word for word in tokens if word.isalpha()]
        # Remove stopword
        tokens = [word.lower() for word in tokens]
        # Lemmatize
        tokens = [word for word in tokens if not word in stopwords]
        tokens = [word for word in tokens if not word in additional]
        tokens = [word for word in tokens if not word in words]
        lemma = WordNetLemmatizer()
        tokens = [lemma.lemmatize(word, pos="v") for word in tokens]
        tokens = [lemma.lemmatize(word, pos="n") for word in tokens]
        
        # list to string
        text = " ".join(tokens)
        
        
        return text


In [6]:
def nouns_extract(text):
        nouns = []  # empty to array to hold all nouns
        text = StringToList(text)
        for word, pos in nltk.pos_tag(text):
            if (pos == 'NN' or pos == 'NNP' or pos == 'NNS' or pos == 'NNPS'):
                nouns.append(word)
                string_nouns = ListToString(nouns)
        return string_nouns

def verbs_extract(text):
    verbs=[]
    text=StringToList(text)
    string_verbs=''
    for word, pos in nltk.pos_tag(text):
        if (pos=='VB' or pos=='VBP' or pos=='VBZ'):
            verbs.append(word)
            string_verbs=ListToString(verbs)
    return string_verbs

In [7]:

    
    # remove useless strings in content and title
df_speech['clean_content']=[clean_text(str(i), additional, words, stopwords) for i in df_speech["content"].values.tolist()]
df_speech['content_nouns']=[nouns_extract(i) for i in df_speech['clean_content'].values.tolist()]
df_speech['content_verbs']=[verbs_extract(i) for i in df_speech['clean_content'].values.tolist()]
    
    
    # create obama_clean.csv for topic modelling and sentimental analysis
df_speech.to_csv('obama_clean.csv',index=False)

if __name__ == '__main__':
    main()